In [6]:
import awswrangler as wr
print(wr.__version__)
print(hasattr(wr, "athena"))
print(hasattr(wr.athena, "read_sql_query"))

3.13.0
True
True


In [2]:
#!/usr/bin/env python3
"""
Script to detect ID columns in your tables
"""
import sys
import os
sys.path.append('.')

from database.connection import DatabaseConnection
import pandas as pd
import re

def analyze_table_columns():
    """Analyze all tables to find ID column patterns"""
    
    tables_to_check = ['institution', 'geography', 'sector', 'instrument', 'gender', 'data_source', 'recipient']
    
    print("ANALYZING ID COLUMNS IN TABLES")
    print("=" * 60)
    
    for table_name in tables_to_check:
        print(f"\nTable: {table_name}")
        print("-" * 30)
        
        try:
            # Get sample data
            df = DatabaseConnection.get_table_data(table_name, limit=5)
            
            if df.empty:
                print("  No data found")
                continue
            
            print(f"  Total columns: {len(df.columns)}")
            
            # Look for ID-like columns
            id_columns = []
            for col in df.columns:
                col_lower = col.lower()
                if any(pattern in col_lower for pattern in ['id', 'key']):
                    id_columns.append(col)
            
            print(f"  ID-like columns: {id_columns}")
            
            # For each ID column, analyze the data
            for id_col in id_columns:
                print(f"    {id_col}:")
                
                # Check data type and sample values
                non_null_values = df[id_col].dropna()
                if len(non_null_values) > 0:
                    print(f"      Sample values: {list(non_null_values.head())}")
                    
                    # Check if numeric
                    try:
                        numeric_values = pd.to_numeric(non_null_values, errors='coerce')
                        if not numeric_values.isna().all():
                            print(f"      Min: {numeric_values.min()}, Max: {numeric_values.max()}")
                            print(f"      Unique values: {len(numeric_values.unique())}/{len(numeric_values)}")
                        else:
                            print("      Non-numeric values")
                    except:
                        print("      Could not analyze as numeric")
                else:
                    print("      All null values")
            
            # Show all columns for reference
            print(f"  All columns: {list(df.columns)}")
            
        except Exception as e:
            print(f"  Error: {str(e)}")

def test_id_detection_algorithm():
    """Test the ID detection algorithm on actual data"""
    print("\n" + "=" * 60)
    print("TESTING ID DETECTION ALGORITHM")
    print("=" * 60)
    
    tables = ['institution', 'geography', 'sector']
    
    for table_name in tables:
        try:
            df = DatabaseConnection.get_table_data(table_name, limit=10)
            
            if df.empty:
                continue
            
            print(f"\nTable: {table_name}")
            
            # Test our ID detection logic
            from database.connection import IDManager
            
            detected_id = IDManager.find_id_column(df, table_name)
            print(f"  Detected ID column: {detected_id}")
            
            if detected_id:
                next_id = IDManager.get_next_id(df, detected_id)
                print(f"  Next ID would be: {next_id}")
                
                # Validate the column
                is_valid, issues = IDManager.validate_ids(df, table_name)
                print(f"  Validation: {'PASS' if is_valid else 'FAIL'}")
                if issues:
                    print(f"  Issues: {issues}")
            
        except Exception as e:
            print(f"Error testing {table_name}: {e}")

if __name__ == "__main__":
    analyze_table_columns()
    test_id_detection_algorithm()

ANALYZING ID COLUMNS IN TABLES

Table: institution
------------------------------
  Total columns: 12
  ID-like columns: ['id_institution']
    id_institution:
      All null values
  All columns: ['id_institution', 'last_verified', 'institution_cpi', 'institution_cpi_short', 'institution_type_layer1', 'institution_type_layer2', 'institution_type_layer3', 'double_counting_risk', 'country_sub', 'country_parent', 'contact_info', 'comments']

Table: geography
------------------------------


2025-10-16 12:49:15.197 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:15.419 
  command:

    streamlit run /opt/homebrew/Caskroom/miniforge/base/envs/cpi-data/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-10-16 12:49:15.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:15.420 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:15.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:15.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:15.423 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


  No data found

Table: sector
------------------------------
  Total columns: 14
  ID-like columns: ['sector_key']
    sector_key:
      Sample values: ['AF_AG_PR', 'AF_AG_SC', 'AF_AG_FS', 'AF_AG_MI', 'AF_AG_AD']
      Non-numeric values
  All columns: ['sector_key', 'sector', 'c1', 'sub_sector', 'c2', 'solution', 'c3', 're', 'ff', 'ee', 'og', 'lt', 'mi', 'ad']

Table: instrument
------------------------------


2025-10-16 12:49:18.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:18.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:18.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:18.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:18.432 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:18.433 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


  No data found

Table: gender
------------------------------


...

  No data found

Table: recipient
------------------------------


2025-10-16 12:49:22.684 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:22.686 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:22.687 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:22.691 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:22.692 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:22.692 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


  No data found

TESTING ID DETECTION ALGORITHM

Table: institution
Error testing institution: cannot import name 'IDManager' from 'database.connection' (/Users/christinalee/repos/ref-frontend/database/connection.py)


2025-10-16 12:49:25.690 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:25.692 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:25.693 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:25.696 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:25.697 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 12:49:25.698 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.



Table: sector
Error testing sector: cannot import name 'IDManager' from 'database.connection' (/Users/christinalee/repos/ref-frontend/database/connection.py)


In [1]:
#!/usr/bin/env python3
"""
Test script for the fixed single-file table insertion logic
"""
import sys
import os
sys.path.append('.')

from database.connection import DatabaseConnection
from datetime import datetime
import boto3

def check_table_structure():
    """Check the actual table structure in S3"""
    print("Checking table structure in S3...")
    
    s3_client = boto3.client('s3', region_name='us-east-1')
    bucket = 'cpi-uk-us-datascience-stage'
    
    # Check institution table file
    institution_key = 'auxiliary-data/reference-data/reference-db/institution/data.parquet'
    
    try:
        response = s3_client.head_object(Bucket=bucket, Key=institution_key)
        print(f"Institution table file exists: s3://{bucket}/{institution_key}")
        print(f"  Size: {response['ContentLength']} bytes")
        print(f"  Last modified: {response['LastModified']}")
        return True
    except Exception as e:
        print(f"Institution table file not found: {e}")
        return False

def read_current_data():
    """Read current data from the institution table"""
    print("\nReading current institution data...")
    
    try:
        df = DatabaseConnection.get_table_data('institution', limit=5)
        print(f"Current row count: {len(df) if not df.empty else 0}")
        
        if not df.empty:
            print("Sample records:")
            print(df[['institution_cpi', 'institution_type_layer1', 'country_sub']].to_string(index=False))
        else:
            print("No data found")
            
        return df
    except Exception as e:
        print(f"Error reading data: {e}")
        return None

def test_single_insert():
    """Test inserting a single record using the new method"""
    print("\n" + "="*50)
    print("TESTING SINGLE INSERT")
    print("="*50)
    
    # Create unique test data
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    test_data = {
        'institution_cpi': f'Test Institution {timestamp}',
        'institution_type_layer1': 'Private',
        'institution_type_layer2': 'Corporation',
        'institution_type_layer3': 'Corporate',
        'country_sub': 'United States',
        'country_parent': 'United States',
        'last_verified': datetime.now().year
    }
    
    print(f"Inserting: {test_data['institution_cpi']}")
    
    # Get row count before
    before_df = DatabaseConnection.get_table_data('institution')
    before_count = len(before_df) if not before_df.empty else 0
    print(f"Rows before insert: {before_count}")
    
    # Attempt insert
    success = DatabaseConnection.execute_insert('institution', test_data)
    
    if success:
        print("Insert operation completed successfully")
        
        # Check row count after
        print("Waiting 3 seconds for consistency...")
        import time
        time.sleep(3)
        
        after_df = DatabaseConnection.get_table_data('institution')
        after_count = len(after_df) if not after_df.empty else 0
        print(f"Rows after insert: {after_count}")
        
        if after_count > before_count:
            print(f"Success! Added {after_count - before_count} row(s)")
            
            # Try to find our specific record
            search_query = f"""
            SELECT * FROM institution 
            WHERE institution_cpi = '{test_data['institution_cpi']}'
            """
            result = DatabaseConnection.execute_query(search_query)
            
            if not result.empty:
                print("Record found in table:")
                print(result.to_string(index=False))
                return True
            else:
                print("Record not found when searching specifically")
                return False
        else:
            print("Row count did not increase")
            return False
    else:
        print("Insert operation failed")
        return False

def test_bulk_insert():
    """Test bulk insert of multiple records"""
    print("\n" + "="*50)
    print("TESTING BULK INSERT")
    print("="*50)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    test_data_list = [
        {
            'institution_cpi': f'Bulk Test Institution 1 {timestamp}',
            'institution_type_layer1': 'Public',
            'institution_type_layer2': 'Government',
            'country_sub': 'Canada',
            'country_parent': 'Canada',
            'last_verified': datetime.now().year
        },
        {
            'institution_cpi': f'Bulk Test Institution 2 {timestamp}',
            'institution_type_layer1': 'Private',
            'institution_type_layer2': 'Funds',
            'institution_type_layer3': 'Venture Capital Fund',
            'country_sub': 'United Kingdom',
            'country_parent': 'United Kingdom',
            'last_verified': datetime.now().year
        }
    ]
    
    print(f"Inserting {len(test_data_list)} records")
    
    # Get row count before
    before_df = DatabaseConnection.get_table_data('institution')
    before_count = len(before_df) if not before_df.empty else 0
    print(f"Rows before bulk insert: {before_count}")
    
    # Attempt bulk insert
    success = DatabaseConnection.bulk_insert('institution', test_data_list)
    
    if success:
        print("Bulk insert operation completed successfully")
        
        # Check row count after
        print("Waiting 3 seconds for consistency...")
        import time
        time.sleep(3)
        
        after_df = DatabaseConnection.get_table_data('institution')
        after_count = len(after_df) if not after_df.empty else 0
        print(f"Rows after bulk insert: {after_count}")
        
        if after_count >= before_count + len(test_data_list):
            print(f"Success! Added {after_count - before_count} row(s)")
            return True
        else:
            print("Expected row count increase not found")
            return False
    else:
        print("Bulk insert operation failed")
        return False

def main():
    print("TESTING SINGLE-FILE TABLE INSERTION")
    print("="*60)
    
    # Check basic structure
    structure_ok = check_table_structure()
    if not structure_ok:
        print("❌ Table structure check failed")
        return
    
    # Read current data
    current_data = read_current_data()
    if current_data is None:
        print("❌ Could not read current data")
        return
    
    # Test single insert
    single_success = test_single_insert()
    
    # Test bulk insert
    bulk_success = test_bulk_insert()
    
    # Summary
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    
    if single_success and bulk_success:
        print("🎉 ALL TESTS PASSED!")
        print("Your insert logic is now working correctly.")
    elif single_success:
        print("✅ Single insert works, ❌ bulk insert failed")
    elif bulk_success:
        print("❌ Single insert failed, ✅ bulk insert works")
    else:
        print("❌ Both tests failed")
        print("Check the error messages above for debugging")

if __name__ == "__main__":
    main()

TESTING SINGLE-FILE TABLE INSERTION
Checking table structure in S3...
Institution table file exists: s3://cpi-uk-us-datascience-stage/auxiliary-data/reference-data/reference-db/institution/data.parquet
  Size: 893986 bytes
  Last modified: 2025-10-07 18:25:31+00:00

Reading current institution data...
Current row count: 5
Sample records:
         institution_cpi institution_type_layer1              country_sub
100% RE IPP GmbH & Co KG                 Private                  Germany
           123Venture SA                 Private                   France
          127 Energy LLC                 Private United States of America
   174 Power Global Corp                 Private United States of America
         1st Source Bank                 Private United States of America

TESTING SINGLE INSERT
Inserting: Test Institution 20251016_121436
Rows before insert: 31275
Starting insert for table: institution
Reading existing data from s3://cpi-uk-us-datascience-stage/auxiliary-data/reference

2025-10-16 12:14:54.789 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


Successfully updated: s3://cpi-uk-us-datascience-stage/auxiliary-data/reference-data/reference-db/institution/data.parquet
Successfully added 1 row to institution table
Cleared Streamlit cache
Insert operation completed successfully
Waiting 3 seconds for consistency...
Rows after insert: 62551
Success! Added 31276 row(s)
Record found in table:
id_institution  last_verified                  institution_cpi institution_cpi_short institution_type_layer1 institution_type_layer2 institution_type_layer3 double_counting_risk   country_sub country_parent contact_info comments
          None           2025 Test Institution 20251016_121436                  None                 Private             Corporation               Corporate                 None United States  United States         None     None

TESTING BULK INSERT
Inserting 2 records
Rows before bulk insert: 62551
Starting bulk insert of 2 rows for table: institution
Reading existing data from s3://cpi-uk-us-datascience-stage/auxiliary-

/Users/christinalee/repos/ref-frontend/database/connection.py:268: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([existing_df, new_rows_df], ignore_index=True)


Created backup: s3://cpi-uk-us-datascience-stage/auxiliary-data/reference-data/reference-db/institution/data.parquet.backup.20251016_121541


2025-10-16 12:15:43.484 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


Successfully updated: s3://cpi-uk-us-datascience-stage/auxiliary-data/reference-data/reference-db/institution/data.parquet
Successfully added 2 rows to institution table
Cleared Streamlit cache
Bulk insert operation completed successfully
Waiting 3 seconds for consistency...
Rows after bulk insert: 93829
Success! Added 31278 row(s)

SUMMARY
🎉 ALL TESTS PASSED!
Your insert logic is now working correctly.
